In [1]:
pip install nussl

     |████████████████████████████████| 239 kB 4.1 MB/s 
     |████████████████████████████████| 51 kB 72 kB/s 
     |████████████████████████████████| 503 kB 44.1 MB/s 
     |████████████████████████████████| 3.3 MB 25.1 MB/s 
     |████████████████████████████████| 232 kB 67.2 MB/s 
     |████████████████████████████████| 146 kB 62.4 MB/s 
     |████████████████████████████████| 69 kB 7.7 MB/s 
     |████████████████████████████████| 87 kB 8.2 MB/s 
     |████████████████████████████████| 963 kB 60.4 MB/s 
     |████████████████████████████████| 129 kB 59.7 MB/s 
     |████████████████████████████████| 6.2 MB 15.0 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=595dfa882ace0af28a65f8d3ab57939bbb0a841125f7feaed4afe9f5f01b137e
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for jams: filename=jams-0.3.4-py3-none-any.whl size=64924 sha256=36c4636cfaaa5c7f52bcc79eb1215a46ce4

In [2]:
# Lecture de musdb (extraits de 7 s.) et packages
import nussl
from IPython.display import Audio
from IPython.display import display as AudioDisplay
from matplotlib import pyplot as plt
import musdb as musdb_package

import numpy as np

musdb = nussl.datasets.MUSDB18(download=True)
mus = musdb_package.DB(download=True)

print('nombre de pistes lues',len(mus))

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


Done!
Done!
nombre de pistes lues 144


In [4]:
# Classification : Création de la target Y
# La cible est la fonction de masque voix (par seuil sur niveau  de psd du spectrogramme)
# Il s'agit de la target à prédire
# En entrée : une piste voix 
# En sortie : le masque temporel 0/1 : shape= (nhop,)
# Je concatène tous ces masques à la suite dans la target y

# numéro/listes des pistes (train/test)

ntrain=20     # nombre de pistes pour la base de train
ntest=5     # nombre de pistes pour la base de test
if(ntrain+ntest>len(mus)):
  print('trop de pistes demandées')

pistes_train=[i for i in range(ntrain)]
pistes_test=[i for i in range(ntrain,ntrain+ntest)]
 


pistes=pistes_train+pistes_test

# paramètres de la STFT
window_length=2048
hop_length=512

# coupure en db (par rapport au max)
db_cutoff = -15. 

# plot?
plot=False

# target 
y_train=np.array([])
y_test=np.array([])

# on tourne sur les pistes choisies
for npiste in pistes:

  voice=musdb[npiste]['sources']["vocals"]

  voice.stft_params = nussl.STFTParams(window_length=window_length, hop_length=hop_length)

  stft = voice.stft()

  # niveau puissance en db (en relatif par rapport max.)
  psd = 10*np.log10(voice.power_spectrogram_data/voice.power_spectrogram_data.max())

  # je moyenne sur canaux et on chope le max dans toute la gamme fréquentielle
  psdmax=np.max( (psd[:,:,0]+psd[:,:,1])/2., axis=0)

  # puis masque basé sur cette valeur
  tmask=np.where(psdmax  < db_cutoff, 0., 1.)
  
  # temps morceau et taille dt (nhop) de la stft
  t=voice.audio_data.shape[1]/voice.sample_rate 
  nhop=tmask.shape[0]
  dt=t/nhop
  t= dt*np.arange(nhop) 
# nfreq est le nombre de fréquences de la stft
  nfreq0=psd.shape[0]

#  print(npiste,nhop,nfreq)

# on trace... 0= pas de voix
  if (plot):
    AudioDisplay(Audio(data=musdb[npiste]['sources']["vocals"].audio_data, rate=musdb[npiste]['sources']["vocals"].sample_rate)) 
    plt.figure(figsize=(12,6))
    plt.subplot(211)
    nussl.core.utils.visualize_spectrogram(voice,y_axis='linear')
    plt.title('Spectro voix')
    plt.ylim(0,6000)
    plt.xlim(0,7)
    plt.subplot(212)
    plt.title('Masque voix')
    plt.scatter(t,y=tmask)
    plt.xlim(0,7)
    plt.yticks([0,1],['Pas de voix', 'Voix'])
    plt.tight_layout()
    plt.show()

# on stocke à la suite toutes ces valeurs
  if (npiste in pistes_train):
    y_train=np.append(y_train,tmask)  
  else:
    y_test=np.append(y_test,tmask)


# ca y est
print('Npiste (train/test)',len(pistes_train),len(pistes_test))
print('Nhop=',nhop)
print('Nfreq=',nfreq0)
print('Shape(train/test)',y_train.shape,y_test.shape)

Npiste (train/test) 20 5
Nhop= 587
Nfreq= 1025
Shape(train/test) (11740,) (2935,)


In [29]:
# # Classification : Création des data X
# --------------------------------------------------------------
# On vient de créer la target (vecteur 1D de taille nhop*npiste)
# A chacune des ces instances temporelles, on y associe sa "tranche" de spectre (qui va donc être les features X, de dimension nfreq)
# Attention, cette fois on travaille sur le spectre du mix (et pas de la voix seule)
# NB. Comme il n'y a rien en HF, on ne garde que les fréquences les plus basses pour alléger 
# Par ex. diviser nfreq par deux, permet de garder la moitié, donc jusque vers 10 kHz ( rappel : f sampling à 44 kHz)
# On crée donc le tableau X de dimension (nhop*npiste, nfreq)

# sous-échantillonnage (on ne prend que la première moitié des fréquences)
nfreq=int(nfreq0/2) 
 
# on tourne sur les pistes choisies

ktrain=0
ktest=0
for npiste in pistes:
  
  mix=musdb[npiste]['mix']
  
  mix.stft_params = nussl.STFTParams(window_length=window_length, hop_length=hop_length)
  stft = mix.stft()

# on passe mono (on moyenne les deux canaux) et cette fois, on passe en amplitude
# et on rescale par la valeur max 

  mono=(stft[:nfreq,:,0]+stft[:nfreq,:,1])/2.
  mono=abs(mono)
  mono=mono/mono.max()
 

# on stocke toutes les données à la suite dans X (on sépare train/test)

  if (npiste in pistes_train):
     # c'est moche mais j'arrive pas à initialiser X....
    ktrain=ktrain+1
    if ktrain==1:
      X_train=mono  
    else:
      X_train=np.append(X_train,mono,axis=1)
   
  if (npiste in pistes_test):
    ktest=ktest+1
    if ktest==1:
      X_test=mono  
    else:
     X_test=np.append(X_test,mono,axis=1)

# par contre X à la fréquence en première composante, on transpose
X_train=X_train.T
X_test=X_test.T
# on vérifie l'accord des shapes
print('train',X_train.shape,y_train.shape)
print('test',X_test.shape,y_test.shape)

train (11740, 512) (11740,)
test (2935, 512) (2935,)


In [72]:
# Classification à proprement parler
# ------------------------------------
import matplotlib.pyplot as plt
from sklearn import ensemble
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report 
from sklearn import neighbors
import xgboost as xgb

# Contextualisation :
# Rappel : Pour chaque frame temporelle : on a nfreq features
# on peut essayer de mettre du contexte (par ex. frame précédent pour améliorer les perfos)
# par ex. en rajoutant la tranche précédente on aura nfreq*2 feature etc.

# stan : j'en suis là....(erreur)

X_train_cont[0,:]=X_train[10:12,:].flatten()
#plt.scatter(range(2*nfreq),X_train_cont)
#plt.show()
#print(X_train_cont.shape)


# choix du classificateur (hyperparamètres à optimiser)
# ----------------------------------------------------
#clf=LogisticRegression(C=1,max_iter=1000)
#clf = neighbors.KNeighborsClassifier(n_neighbors=7, metric='minkowski')

clf=ensemble.RandomForestClassifier(n_jobs=-1, random_state=321, n_estimators=100)


clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print('CT RF',pd.crosstab(y_test,y_pred))
print('score RF',clf.score(X_test,y_test))



#essais RF (20/5)-
#  ref          : 0.828
#  context -1 :

#XGB (en cours)
#train = xgb.DMatrix(data=X_train, label=y_train)
#test = xgb.DMatrix(data=X_test, label=y_test)
#params = {'booster': 'gbtree', 'learning_rate': 0.2, 'objective': 'binary:logistic'}
#xgb = xgb.train(params=params, dtrain=train, num_boost_round=10, evals=[(train, 'train'), (test, 'eval')])
#preds = xgb.predict(test)
#xgbpreds = pd.Series(np.where(preds > 0.5, 1, 0))
#print('CT XGB',pd.crosstab(xgbpreds, pd.Series(y_test)))
#print('XGB score',xgb.eval(test))

# petite comparaison graphique
#t= dt*np.arange(len(y_test)) 
#plt.figure(figsize=(10,6))
#plt.scatter(t,y_test,s=50,alpha=0.1)
#plt.scatter(t,y_pred,color='red',s=50,alpha=0.1)
#plt.show


IndexError: ignored